In [ ]:
from pyspark.sql import SparkSession
import  pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *

columns = StructType([
            StructField('order_id',IntegerType(),True),
            StructField('customer_id',StringType(),True),
            StructField('order_date',StringType(),True),
            StructField('amount',IntegerType(),True),
            StructField('region',StringType(),True),
            StructField('product',StringType(),True)
            ])
data = [
        (1, 'C1', '2024-01-01', 500, 'South', 'Laptop'),
        (2, 'C1', '2024-01-05', 700, 'South', 'Mobile'),
        (3, 'C2', '2024-01-03', 300, 'North', 'Mobile'),
        (4, 'C2', '2024-01-10', 900, 'North', 'Laptop'),
        (5, 'C3', '2024-01-02', 400, 'South', 'Tablet'),
        (6, 'C3', '2024-01-08', 600, 'South', 'Laptop'),
        (7, 'C4', '2024-01-04', 200, 'East', 'Mobile'),
        (8, 'C4', '2024-01-09', 500, 'East', 'Tablet'),
        (9, 'C5', '2024-01-06', 800, 'West', 'Laptop'),
        (10, 'C5', '2024-01-12', 1200, 'West', 'Laptop'),
        (10, 'C6', '2024-01-12', 1200, 'West', 'Laptop')
        ]

spark = SparkSession.builder.appName("test").getOrCreate()
df = spark.createDataFrame(data=data,schema= columns)

df = df.withColumn('order_date',F.col('order_date').cast('date'))

#Find total sales per customer.
df.groupBy('customer_id').agg(F.sum(F.col("amount"))).show()

# # #Find total sales per region.
df.groupBy('region').agg(F.sum(F.col("amount"))).show()

# #Find the highest order amount per region.
df.groupBy('region').agg(F.max(F.col("amount"))).show()

# # Find the top-spending customer in each region
win = Window.partitionBy('region').orderBy(F.col('amount').desc())

df.withColumn('highest_per_region',F.dense_rank().over(win))\
   .filter(F.col('highest_per_region')==1)\
   .select('customer_id','amount','region').show()

# # Find customers who placed more than 1 order.
df.groupBy('customer_id').agg(F.count('*').alias('number_order'))\
  .filter(F.col('number_order')>1)\
  .show()

# # For each customer, find the first and last order date.
df.groupBy('customer_id')\
    .agg(F.min(F.col("order_date")).alias("first_order"),\
    F.max(F.col("order_date")).alias("last_order"))\
    .show()

# Find running total of sales per customer ordered by order_date
win_spec = Window.partitionBy('customer_id').orderBy('order_date')
df.withColumn('running total',F.sum('amount').over(win_spec))\
    .select('customer_id','order_date','amount','running total')\
    .show()


# Q🔟 (Very Common Interview Question)
# Find customers whose total spending is greater than the average spending of all customers.

df1 =df.agg(F.avg('amount').cast('Int').alias("avg_amount"))
df2 =df.groupBy('customer_id').agg(F.sum('amount').alias("total_spend"))
df2.filter(df2["total_spend"]>df1["avg_amount"]).show()

# Q9️⃣
# Add a column order_category:
# High → amount >= 800
# Medium → 400–799
# Low → < 400

df.withColumn('order_category',F.when(F.col('amount')>=800,'High')\
                              .when(((F.col('amount')<=799) | (F.col('amount')>=400)),'medium')\
                              .otherwise('low'))\
                              .show()
       

In [ ]:
#Find customers whose total spend is greater than 800
df.groupBy('customer_id').agg(F.sum('amount').alias("total_spend"))\
  .filter(F.col('total_spend')>800)\
  .show()

In [ ]:
# Add a column order_rank to rank orders per customer by amount (highest first) using window functions.

win_speci = Window.partitionBy('customer_id').orderBy(F.col('amount').desc())

df.withColumn("order_rank",F.dense_rank().over(win_speci)).show()

In [ ]:
from pyspark.sql import functions as F

df.groupBy("customer_id") \
  .agg(F.sum("amount").alias("total_spend")) \
  .orderBy(F.col("total_spend").desc()) \
  .limit(1) \
  .show()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

spark = SparkSession.builder.appName("consecutive_days").getOrCreate()

data = [
    (1, "C1", "2024-01-01 10:00:00"),
    (2, "C1", "2024-01-02 11:00:00"),
    (3, "C1", "2024-01-03 09:30:00"),  # ✅ 3 consecutive days

    (4, "C2", "2024-01-01 08:00:00"),
    (5, "C2", "2024-01-03 12:00:00"),
    (6, "C2", "2024-01-04 13:00:00"),  # ❌ not consecutive (missing Jan 2)

    (7, "C3", "2024-01-05 10:00:00"),
    (8, "C3", "2024-01-06 10:30:00"),
    (9, "C3", "2024-01-07 11:00:00"),
    (10,"C3", "2024-01-08 09:00:00")   # ✅ 4 consecutive days
]

schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_ts", StringType(), True)
])

df = spark.createDataFrame(data, schema)

df = df.withColumn("order_ts", F.to_timestamp("order_ts"))

df.show(truncate=False)
df.printSchema()


In [ ]:
# Step 1: get distinct order dates per customer
df_distinct = df.select(
    "customer_id",
    F.to_date("order_ts").alias("order_date")
)
# Step 2: assign row_number per customer ordered by date
win = Window.partitionBy("customer_id").orderBy("order_date")

df_rn = df_distinct.withColumn("rn", F.row_number().over(win))

# Step 3: date - rn trick
df_group = df_rn.withColumn(
    "grp_date",
    F.date_sub(F.col("order_date"), F.col("rn"))
)
df_group.show()

# Step 4: find streaks of 3 or more days
result = df_group.groupBy("customer_id", "grp_date") \
    .count() \
    .filter(F.col("count") >= 3) \
    .select("customer_id") \
    .distinct()

result.show()

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

data = [
    (1, "U1", "C1", "2024-01-01 10:00:00", 500, "SUCCESS"),
    (2, "U1", "C1", "2024-01-01 10:20:00", 300, "SUCCESS"),
    (3, "U1", "C1", "2024-01-01 11:00:00", 200, "FAILED"),
    (4, "U2", "C2", "2024-01-02 09:00:00", 700, "SUCCESS"),
    (5, "U2", "C2", "2024-01-02 09:25:00", 400, "SUCCESS"),
    (6, "U2", "C2", "2024-01-02 10:30:00", 100, "SUCCESS"),
    (7, "U3", "C3", "2024-01-03 12:00:00", 900, "FAILED"),
    (8, "U3", "C3", "2024-01-03 12:40:00", 800, "SUCCESS"),
    (9, "U3", "C3", "2024-01-03 13:30:00", 600, "SUCCESS"),
    (10,"U1", "C1", "2024-01-02 10:00:00", 1000, "SUCCESS")
]

columns = [
    "txn_id",
    "user_id",
    "customer_id",
    "txn_time",
    "amount",
    "status"
]

spark = SparkSession.builder.appName("test").getOrCreate()

df = spark.createDataFrame(data=data,schema=columns)

df = df.withColumn("txn_id",F.col("txn_id").cast("Int"))
df = df.withColumn("user_id",F.col("user_id").cast("String"))
df = df.withColumn("customer_id",F.col("customer_id").cast("String"))
df = df.withColumn("txn_time",F.to_timestamp(F.col("txn_time")))
df = df.withColumn("amount",F.col("amount").cast("Int"))
df = df.withColumn("status",F.col("status").cast("String"))


+------+-------+
|txn_id|user_id|
+------+-------+
+------+-------+



In [5]:
#Find users who have at least 2 consecutive SUCCESS transactions based on transaction time.

# Window per user ordered by transaction time
win = Window.partitionBy("user_id").orderBy("txn_time")

# Get previous transaction status
df2 = df.withColumn("prev_status", F.lag("status").over(win))

# Find consecutive SUCCESS transactions
result = (
    df2
    .filter((F.col("status") == "SUCCESS") & (F.col("prev_status") == "SUCCESS"))
    .select("user_id")
    .distinct()
)

result.show()

+-------+
|user_id|
+-------+
|     U1|
|     U2|
|     U3|
+-------+

